# AEROSP 536 Electric Propulsion: Homework 3
Jason Chen

## Problem 1
### Part (a)

## Problem 5
### Part (a)

In this magnetic nozzle problem, we are asked to find the heater, frozen flow, and divergence efficiencies. Starting with the heater efficiency $\eta_h$, we know that generally:

$$ \eta_h = 1-\frac{\dot{Q}_{loss}}{P_{in}} $$

where $\dot{Q}_{loss}$ is the total heat lost in the heater/process as opposed to the power input $P_{in}$. From the problem statement, we know the lost heat is dominated by transfer across the back wall area, which is given by:
$$
\dot{Q}_{wall} = \frac{n_e (k_B T_e)^{3/2} A_{wall}}{4\sqrt{m_i}} \left[ \frac{5}{2}+2\ln{\sqrt{\frac{m_i}{m_e}}} \right]
$$
The only unknown in this equation (others are known constants or given in the problem statement) is $n_e$, which is the electron particle density in the chamber. We can find this using the given information: the electron and ion mole fractions are equal ($n_i = n_e$), inlet mass flow rate is known, and noting the continuity equation:
$$
\dot{m} = \rho A v
$$
We know the neutral mass flow into the engine, given as a standard volumetric flow rate. To convert 1 sccm to a mass flow rate, we multiply by the density of xenon at standard temperature and pressure (for US units, 68 Fahrenheit and 1 atm).
$$
0.7 \times \dot{m}_{in} = \dot{m}_{out} = \textrm{SCCM} \times \rho_{STP}
$$
From the statement that the ions begin with the thermal velocity at 300 K and the chamber diameter, we can solve for the mass density of the ions, $\rho_i$. The thermal velocity is given by:
$$
u_{thermal} = \sqrt{\frac{8k_B T}{\pi m_n}}
$$
Finally, with $\rho_i$, we can convert to a particle density by dividing by the molar weight and using Avogadro's number:
$$
n_e = n_i = [\rho_i / (0.1313 \textrm{ kg/mol})](6.02 \times 10^{23}) 
$$


In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pint import UnitRegistry
ureg = UnitRegistry(auto_reduce_dimensions=True)
Q_ = ureg.Quantity

k_B = Q_(1.38e-23, 'J/K')  # Boltzmann constant
m_n = Q_(131.3, 'amu')  # Mass of xenon atom
T = Q_(300, 'K')  # Thermal velocity temperature
rho_stp = Q_(5.8965, 'kg/m^3')  # Xenon density at STP
mdot_in = 0.7 * Q_(1, 'cm^3/min') * rho_stp
u_thermal = np.sqrt((8 * k_B * T)/(np.pi * m_n))
A_chamber = np.pi * (Q_(2, 'cm')**2) / 4
rho_i = mdot_in / (u_thermal * A_chamber)
n_e = (rho_i / Q_(131.3, 'g/mol')) * Q_(6.022e23, '1/mol')
print(f"Electron number density, n_e: {n_e.m_as('1/m^3'):.2e} 1/m^3")

Electron number density, n_e: 4.57e+18 1/m^3


Plugging into the $\dot{Q}_{wall}$ equation (while searching up the mass of an electron):

In [43]:
T_e = Q_(10, 'eV')/Q_(8.617e-5, 'eV/K')
constant = 5/2 + 2 * np.log(np.sqrt(m_n / Q_(9.12e-31, 'kg')))
Q_dot_wall = ((n_e * (k_B * T_e)**(3/2) * A_chamber) / (4 * np.sqrt(m_n))) * constant
P_in = Q_(30, 'W')
print(f"Heat flow rate at wall: {Q_dot_wall.m_as('W'):.1f} W")
eta_h = (1 - Q_dot_wall / P_in)
print(f"Heating efficiency: {eta_h.m_as('dimensionless'):.2%}")


Heat flow rate at wall: 23.2 W
Heating efficiency: 22.75%


To find the divergence efficiency, it is simple trigonometry (note we use the half-angle):
$$
\eta_{D} = \cos{\theta_D}^2 = \left(\frac{\sqrt{3}}{2}\right)^2 = 75 \%
$$

To find the frozen flow efficiency, we can start with the general form which is essentially the residual energy over the total available energy:
$$
\eta_f = \frac{(\langle h_c \rangle + \langle \mu_c \rangle) - (\langle h_e \rangle + \langle \mu_e \rangle)}{\langle h_c \rangle + \langle \mu_c \rangle} \approx 1 - \frac{\langle \mu_e \rangle}{\langle h_c \rangle + \langle \mu_c \rangle}
$$
since $T_e << T_c$, so this assumption is valid. In chemical contexts, $\mu_e$ is the residual chemical (stored bond) energy, i.e. how much heat of formation is yet to be released in the exhaust product. In this context, it is analagous to the amount of ions still present in the exhaust times the first ionization energy, since this would be the amount of energy that could have been used to produce momentum for the vehicle, but is instead unaffected by the magnetic nozzle. In other words:
$$
\mu_e = \epsilon_{iz} f_n
$$
where $f_n$ is the mole fraction of neutral atoms at the exit (different from $n_n$ since this is not normalized by volume). Since we know from the problem statement that $\dot{m}_{i,out} = 0.7 \dot{m}_{total,in}$ and that a single neutral Xenon atom produces exactly 1 electron and 1 ion when ionized, then $f_n = 0.3$ and $f_i = f_e = 0.7$. Since we know the relative concentrations of particles does not change, $\mu_e = \mu_c$.

The total available energy in the heating chamber will be the mass (and since flux is constant, mole fraction is analagous) that has been ionized, which is $f_i = f_e$. Assuming a calorically perfect gas (valid at the temperature of 300 K), its energy can be found by $h = c_p T$, or since Xenon is a monoatomic gas with only translational and vibrational modes of energy storage, we get $h_c = \frac{5}{2} f k_B T$ for each neutrals and ions, while electron thermal energy can be approximated as $k_B T_e$, with the initial temperature of electrons in the chamber being given.

Putting it together:
$$
\eta_f = 1 - \frac{\langle \mu_e \rangle}{\langle h_c \rangle + \langle \mu_c \rangle} = 1-\frac{\epsilon_{iz} f_n}{k_B(\frac{5}{2} f_i T_i + \frac{5}{2} f_n T_n + f_e T_e) + \epsilon_{iz} f_n}
$$

In [ ]:
mu = Q_(12.13, 'eV') * 0.3
eta_f = 1-(mu/(k_B*((5/2)*0.7*T + (5/2)*0.3*T + 0.7*T_e) + mu))
print(f"Frozen flow efficiency: {eta_f.m_as('dimensionless'):.2%}")
print(f"Total efficiency: {(eta_h * 0.75 * eta_f).m_as('dimensionless'):.2%}")

Frozen flow efficiency: 65.99%
Total efficiency: 11.26%


Putting it together:
$$
\boxed{\eta = \eta_h \eta_d \eta_f = 11.3 \%}
$$
This is a very low efficiency, but is dominated by the heating efficiency.

### Part (b)
To find specific impulse and thrust, we use the average exit velocity (ideally we would use the integral to find the actual effective velocity, but we don't have enough information here). From lecture:
$$
\langle u_e \rangle = \sqrt{\frac{2 P_{in}}{\dot{m}} \eta}
$$
We get:
$$
\boxed{I_{sp} = \langle u_e \rangle / g_0 = 1010.2 \textrm{ s}}
$$
We will assume matched conditions at the exit (vacuum operation, and the fact that temperature reaches zero at the exit), so no pressure thrust:
$$
\boxed{F_t = \dot{m} \langle u_e \rangle = 0.7 \textrm{ mN}}
$$

In [46]:
u_e = np.sqrt((2 * P_in / mdot_in) * (eta_h * 0.75 * eta_f))
I_sp = u_e / Q_(9.81, 'm/s^2')
thrust = mdot_in * u_e
print(f"Specific Impulse: {I_sp.m_as('s'):.1f} s")
print(f"Thrust: {thrust.m_as('mN'):.3f} mN")

Specific Impulse: 1010.2 s
Thrust: 0.682 mN


### Part (c)
The performance we derived here is much lower than the reported $40 \%$ overall efficiency, and this is likely due to a couple possible factors:
- The heating efficiency seems irregularly low in our derivation, but assuming it's correct, then recent advances in X-ray heating could allow the real device to package the chamber in a smaller area, thereby decreasing heat loss through the walls 
- The ionization process in the real device may be more efficient, as the frozen flow losses come from unionized neutral atoms. It's possible that at higher input powers, the ionization process scales to be more efficient much better than at lower power
- The assumption that there is no recombination of ions and electrons is likely not true, but this would actually lower efficiency since the magnetic nozzle would not be able to fully accelerate that mass. This points to the fact that our efficiency calculation may be flawed in some way